In [4]:
import os
import json


import pandas as pd

In [5]:
sample = {
    "positions": {
        "1.0": {
            "Nose": [0.4727230370044708, 0.3838244676589966, -0.27421602606773376],
            "Left_eye_inner": [0.4753738045692444, 0.37479984760284424, -0.26180776953697205],
            "Left_eye": [0.47714120149612427, 0.3745166063308716, -0.26183751225471497],
            "Left_eye_outer": [0.4789522588253021, 0.37425851821899414, -0.26184573769569397],
            "Right_eye_inner": [0.4696425199508667, 0.37428316473960876, -0.2632329761981964],
            "Right_eye": [0.4674718976020813, 0.37366846203804016, -0.26325973868370056],
            "Right_eye_outer": [0.4651995897293091, 0.37315359711647034, -0.26330235600471497],
            "Left_ear": [0.4796261489391327, 0.3754121959209442, -0.16696897149085999],
            "Right_ear": [0.4597683548927307, 0.3751758933067322, -0.1745608150959015],
            "Mouth_left": [0.475565642118454, 0.39149120450019836, -0.23681190609931946],
            "Mouth_right": [0.4675469994544983, 0.39173346757888794, -0.2391127645969391],
            "Left_shoulder": [0.4979342222213745, 0.42636486887931824, -0.0960470512509346],
            "Right_shoulder": [0.4418589472770691, 0.43039795756340027, -0.09553270041942596],
            "Left_elbow": [0.5008105039596558, 0.4966505765914917, -0.05243808776140213],
            "Right_elbow": [0.4394679069519043, 0.5003064870834351, -0.05494270101189613],
            "Left_wrist": [0.502241313457489, 0.5615555047988892, -0.13062617182731628],
            "Right_wrist": [0.4392949938774109, 0.5611810088157654, -0.13273857533931732],
            "Left_pinky": [0.5023744702339172, 0.5801471471786499, -0.15442657470703125],
            "Right_pinky": [0.44217008352279663, 0.5799118876457214, -0.1556280553340912],
            "Left_index": [0.4990525245666504, 0.5818195939064026, -0.17855951189994812],
            "Right_index": [0.44541043043136597, 0.5800011157989502, -0.18217860162258148],
            "Left_thumb": [0.49780577421188354, 0.5752022862434387, -0.14155635237693787],
            "Right_thumb": [0.44588392972946167, 0.5738919973373413, -0.14443477988243103],
            "Left_hip": [0.4885951280593872, 0.5648671984672546, -0.0021941151935607195],
            "Right_hip": [0.4544827342033386, 0.5639280676841736, 0.00222505209967494],
            "Left_knee": [0.48854485154151917, 0.6586241722106934, -0.02576499618589878],
            "Right_knee": [0.45985522866249084, 0.6649723649024963, -0.01617051102221012],
            "Left_ankle": [0.49369779229164124, 0.7492525577545166, 0.1270214319229126],
            "Right_ankle": [0.4526473581790924, 0.7565955519676208, 0.11720269173383713],
            "Left_heel": [0.49099811911582947, 0.7575002312660217, 0.13673612475395203],
            "Right_heel": [0.45248162746429443, 0.7645536065101624, 0.12494811415672302],
            "Left_foot_index": [0.4925276041030884, 0.7878919243812561, 0.04002412036061287],
            "Right_foot_index": [0.45452722907066345, 0.7990591526031494, 0.018001534044742584]
        },
        "2.0": {
            "Nose": [0.4726940393447876, 0.3831414580345154, -0.26175472140312195],
            "Left_eye_inner": [0.47510889172554016, 0.373349666595459, -0.24973495304584503],
            "Left_eye": [0.47685539722442627, 0.37310147285461426, -0.24976149201393127],
            "Left_eye_outer": [0.4786836504936218, 0.3729214072227478, -0.24976757168769836],
            "Right_eye_inner": [0.46944373846054077, 0.3729451894760132, -0.2508068382740021],
            "Right_eye": [0.4673190414905548, 0.3724323809146881, -0.25082799792289734],
            "Right_eye_outer": [0.46509063243865967, 0.37205004692077637, -0.25087013840675354],
            "Left_ear": [0.4796132743358612, 0.37473800778388977, -0.1597304791212082],
            "Right_ear": [0.45977213978767395, 0.37455999851226807, -0.16519752144813538],
            "Mouth_left": [0.4756295382976532, 0.3907076120376587, -0.22632761299610138],
            "Mouth_right": [0.46772199869155884, 0.3906969428062439, -0.22808308899402618],
            "Left_shoulder": [0.4979526400566101, 0.4263125956058502, -0.09424526989459991],
            "Right_shoulder": [0.4417797923088074, 0.43034741282463074, -0.08744104951620102],
            "Left_elbow": [0.5012014508247375, 0.4964407980442047, -0.04724247008562088],
            "Right_elbow": [0.4388962984085083, 0.5015712380409241, -0.04607902467250824],
            "Left_wrist": [0.5024884939193726, 0.5597358345985413, -0.11459018290042877],
            "Right_wrist": [0.4389660060405731, 0.5606597661972046, -0.1166703924536705],
            "Left_pinky": [0.5031002759933472, 0.5795527696609497, -0.13527444005012512],
            "Right_pinky": [0.4410523474216461, 0.5797632336616516, -0.13701032102108002],
            "Left_index": [0.5001664757728577, 0.5803791284561157, -0.15812329947948456],
            "Right_index": [0.4441104233264923, 0.5790987014770508, -0.1630110740661621],
            "Left_thumb": [0.49856647849082947, 0.5737610459327698, -0.12493623793125153],
            "Right_thumb": [0.4444619417190552, 0.5730152726173401, -0.12801975011825562],
            "Left_hip": [0.48892828822135925, 0.5621697902679443, -0.005039631854742765],
            "Right_hip": [0.45490607619285583, 0.5626218318939209, 0.005029990803450346],
            "Left_knee": [0.4885903000831604, 0.6585646271705627, -0.02346721664071083],
            "Right_knee": [0.46099233627319336, 0.6625020503997803, -0.0035396532621234655],
            "Left_ankle": [0.49394509196281433, 0.7517425417900085, 0.12642322480678558],
            "Right_ankle": [0.45273780822753906, 0.7559921145439148, 0.13847103714942932],
            "Left_heel": [0.4910394847393036, 0.7606337666511536, 0.13554228842258453],
            "Right_heel": [0.45263031125068665, 0.7640327215194702, 0.14737264811992645],
            "Left_foot_index": [0.49276602268218994, 0.7902218103408813, 0.035742588341236115],
            "Right_foot_index": [0.45474597811698914, 0.7973014116287231, 0.04534585028886795]
        },
    }
}

In [15]:
def getFilesPath(directory):
    files = []
    for root, dirs, file in os.walk(directory):
        for f in file:
            files.append(os.path.join(root, f))
    return files


def loadFiles(files_path, activities):
    activity_dfs = {}
    for activity in activities:
        dataframes = []
        for file_path in files_path:
            if file_path.endswith('.json') and activity in file_path:
                with open(file_path, 'r') as file:
                    file_data = json.load(file)
                    file_df = flatternData(file_data)
                    dataframes.append(file_df)
        if dataframes:
            activity_dfs[activity] = pd.concat(dataframes, ignore_index=True)
    return activity_dfs


def flatternData(data):
    flattened_data = {}
    for timestamp, position in data['positions'].items():
        for keypoint, coordinates in position.items():
            flattened_data[f'{timestamp}_{keypoint}_x'] = coordinates[0]
            flattened_data[f'{timestamp}_{keypoint}_y'] = coordinates[1]
            flattened_data[f'{timestamp}_{keypoint}_z'] = coordinates[2]
    return pd.DataFrame([flattened_data])


def calculateMeanForAllVideos(df):
    mean_dfs = []

    for index, row in df.iterrows():
        mean_data = {}
        for column in row.index:
            # Extract joint name and position (e.g., Nose_x)
            joint = '_'.join(column.split('_')[1:])
            if joint not in mean_data:
                mean_data[joint] = []
            mean_data[joint].append(row[column])

        # Calculate the mean for each joint
        # print(mean_data)
        mean_data = {joint: pd.Series(values).mean()
                     for joint, values in mean_data.items()}

        # Create a DataFrame for the current video
        mean_df = pd.DataFrame(mean_data, index=[index])
        mean_dfs.append(mean_df)

    # Concatenate all mean DataFrames into one
    final_mean_df = pd.concat(mean_dfs, ignore_index=True)
    return final_mean_df

In [24]:
# path to data
group1A_path = 'C:\\Users\\PhucNg\\Documents\\KeraalDataset\\Keraal only data\\Group1A\\blazepose'
group2A_path = 'C:\\Users\\PhucNg\\Documents\\KeraalDataset\\Keraal only data\\Group2A\\blazepose'
group3_path = 'C:\\Users\\PhucNg\\Documents\\KeraalDataset\\Keraal only data\\Group3\\blazepose'

# activities
# activities = ['CTK', 'ELK', 'RTK']
activities = ['CTK', 'ELK']

# load data
group1A_files_path = getFilesPath(group1A_path)
group2A_files_path = getFilesPath(group2A_path)
group3_files_path = getFilesPath(group3_path)
group1A_df = loadFiles(group1A_files_path, activities)
group2A_df = loadFiles(group2A_files_path, activities)
# group3_df = loadFiles(group3_files_path, activities)

In [32]:
# Add labels
for activity, df in group1A_mean_df.items():
    df['Backpain'] = 1  # Backpain = 1

for activity, df in group2A_mean_df.items():
    df['Backpain'] = 0  # Backpain = 0

merged_df_by_activity = {
    activity: pd.concat([group1A_mean_df[activity], group2A_mean_df[activity]], ignore_index=True) for activity in activities
}

# Concatenate all DataFrames if needed


# Display the final DataFrame
merged_df_by_activity['CTK']

,Nose_x,Nose_y,Nose_z,Left_eye_inner_x,Left_eye_inner_y,Left_eye_inner_z,Left_eye_x,Left_eye_y,Left_eye_z,Left_eye_outer_x,...,Right_heel_x,Right_heel_y,Right_heel_z,Left_foot_index_x,Left_foot_index_y,Left_foot_index_z,Right_foot_index_x,Right_foot_index_y,Right_foot_index_z,Backpain
0,0.535141,0.417495,-0.241703,0.538110,0.408839,-0.229124,0.539868,0.408634,-0.229144,0.541592,...,0.496740,0.805393,0.056291,0.580529,0.847384,-0.065473,0.479200,0.841782,-0.042025,1
1,0.532774,0.409688,-0.174043,0.535823,0.401459,-0.158861,0.537574,0.401233,-0.158892,0.539343,...,0.496190,0.802403,0.034351,0.580508,0.846465,-0.079738,0.480062,0.839426,-0.063522,1
2,0.531821,0.412390,-0.193253,0.535134,0.404024,-0.178300,0.537083,0.403829,-0.178327,0.539025,...,0.495939,0.802809,0.038622,0.581737,0.847051,-0.076743,0.479515,0.840028,-0.059671,1
3,0.533755,0.410594,-0.176886,0.536870,0.402395,-0.160979,0.538730,0.402222,-0.161005,0.540581,...,0.496245,0.803022,0.019283,0.581675,0.848841,-0.091406,0.479717,0.841013,-0.077819,1
4,0.532220,0.410489,-0.166449,0.535395,0.402150,-0.150432,0.537309,0.402018,-0.150463,0.539258,...,0.496768,0.804154,0.017334,0.581408,0.847669,-0.093451,0.479852,0.841244,-0.079396,1
5,0.516030,0.408906,-0.286127,0.520455,0.396575,-0.271083,0.523314,0.395958,-0.271110,0.526157,...,0.469737,0.890179,0.144046,0.570374,0.949881,0.002500,0.444499,0.937573,0.029858,1
6,0.514496,0.405593,-0.218144,0.518683,0.392847,-0.200311,0.521572,0.392201,-0.200346,0.524483,...,0.472039,0.887250,0.091820,0.570655,0.947678,-0.034628,0.447616,0.938321,-0.019170,1
7,0.515392,0.408350,-0.271714,0.519851,0.394738,-0.255608,0.522732,0.393955,-0.255651,0.525331,...,0.474998,0.872626,0.096495,0.572940,0.928700,-0.009756,0.453269,0.923978,-0.019183,1
8,0.514676,0.404968,-0.231954,0.518959,0.392921,-0.214297,0.521809,0.392359,-0.214322,0.524618,...,0.469487,0.889311,0.123730,0.569726,0.948368,-0.032015,0.442952,0.935934,0.013445,1
9,0.513495,0.404394,-0.226976,0.517619,0.391666,-0.209548,0.520445,0.390994,-0.209574,0.523366,...,0.470301,0.888746,0.114473,0.570037,0.949492,-0.034244,0.442866,0.937762,0.003490,1


In [37]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Assuming merged_df_by_activity is already defined and contains the merged DataFrame for each activity
df_ctk = merged_df_by_activity['CTK']

# Prepare the data
X = df_ctk.drop(columns=['Backpain'])  # Features
y = df_ctk['Backpain']  # Labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Define the parameter grid for the KNeighborsClassifier
param_grid = {
    'n_neighbors': [3, 4, 5],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

# Initialize the KNeighborsClassifier
knn_clf = KNeighborsClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=knn_clf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=3)

# Perform grid search and cross-validation
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Print the best parameters and best score
print(f"Best parameters: {best_params}")
print(f"Best cross-validation score: {grid_search.best_score_}")

# Evaluate the model
y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test set accuracy: {accuracy}")

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best parameters: {'algorithm': 'auto', 'n_neighbors': 5, 'weights': 'uniform'}
Best cross-validation score: 0.4178571428571428
Test set accuracy: 0.2
